# 生活中的优化问题
by Qiao for NLP8 2020-09-26
- 问题描述请结合课件
- 这里使用的solver使用scipy和cvxpy提供的接口。

In [1]:
# !pip install cvxpy cvxopt
import cvxpy as cp
from scipy.optimize import linprog
import numpy as np
import pandas as pd

# 1. 交通运输问题


### a.课程中产销平衡问题的求解

In [2]:


# 目标函数中各参数项的系数 Object = c^T x 
c = [6, 4, 6, 6, 5, 5]

# 等式约束条件：  Ax = b
A = [[1, 1, 1, 0, 0, 0], 
     [0, 0, 0, 1, 1, 1],
     [1, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [0, 0, 1, 0, 0, 1]
    ]
b = [200, 300, 150, 150, 200]

# X的定义域约束：
x_bounds = [(0, None)]*6

res = linprog(c, A_eq=A, b_eq=b, bounds=x_bounds, method='revised simplex')
res

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.


     con: array([0., 0., 0., 0., 0.])
     fun: 2500.0
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 50., 150.,   0., 100.,   0., 200.])

### b.课程中产>销问题的求解

In [3]:


# 目标函数中各参数项的系数 Object = c^T x 
c = [6, 4, 6, 6, 5, 5]

# 等式约束条件：  Ax = b
A = [
     [1, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [0, 0, 1, 0, 0, 1]
    ]
b = [150, 100, 200]

# 不等式约束条件: Gx <= q
G = [
     [1, 1, 1, 0, 0, 0], 
     [0, 0, 0, 1, 1, 1],
    ]
q = [200, 300]

# X的定义域约束：
x_bounds = [(0, None)]*6

res = linprog(c, A_ub=G, b_ub=q, A_eq=A, b_eq=b, bounds=x_bounds, method='revised simplex')
res


     con: array([0., 0., 0.])
     fun: 2300.0
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([ 0., 50.])
  status: 0
 success: True
       x: array([100., 100.,   0.,  50.,   0., 200.])

### c.课程中多物品运输问题的求解

In [4]:


# 目标函数中各参数项的系数 Object = c^T x 
c = [6, 4, 6, 6, 5, 5, 3, 5, 2, 9, 3, 4]

# 等式约束条件：  Ax = b
A = [
     [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    ]
b = [150, 150, 200, 50, 300, 250]

# 不等式约束条件: Gx <= q
G = [
     [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], 
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    ]
q = [200, 300, 150, 500]

# X的定义域约束：
x_bounds = [(0, None)]*12

res = linprog(c, A_ub=G, b_ub=q, A_eq=A, b_eq=b, bounds=x_bounds, method='revised simplex')
res

     con: array([0., 0., 0., 0., 0., 0.])
     fun: 4350.0
 message: 'Optimization terminated successfully.'
     nit: 14
   slack: array([ 0.,  0.,  0., 50.])
  status: 0
 success: True
       x: array([ 50., 150.,   0., 100.,   0., 200.,  50.,   0., 100.,   0., 300.,
       150.])

# 2.整数规划问题

### a.鸡兔同笼


In [5]:
X = cp.Variable(3, integer=True)

objective = cp.Minimize(cp.sum(X[1:]))
constraints = [
               cp.sum(X) == 32, # 头
               cp.sum(X * np.array([2, 4, 8])) == 80, # Jio
               X >= 0
]

prob = cp.Problem(objective, constraints)
res = prob.solve()
print("Min y+z: ", res)
print("x, y, z = ")
print(X.value)

Min y+z:  4.0
x, y, z = 
[28.  2.  2.]


### b.背包问题

In [6]:
X = cp.Variable(4, boolean=True)

objective = cp.Maximize(cp.sum(X * np.array([16, 19, 23, 28])))
constraints = [
               cp.sum(X * np.array([30, 35, 51, 72])) <= 100, # 体积
               cp.sum(X * np.array([2, 3, 4, 5])) <= 7, # 重量
]

prob = cp.Problem(objective, constraints)
res = prob.solve()
print("Max value : ", res)
print("x, y, z, w = ")
print(X.value)

Max value :  42.0
x, y, z, w = 
[0. 1. 1. 0.]


### c.饮食规划

In [7]:

nutrients = ["Calory", "Carbo", "Protein", "VitA", "VitC", "Calc", "Iron"]
food = ["芝士牛肉堡", "马卡龙", "巨无霸", "FFilet", "鸡肉", "炸薯条", "麦满分", "1%LF Milk", "橙汁"]

intake_thred = np.array([
                         (2000, None),
                         (350, 375),
                         (55, None),
                         (100, None),
                         (100, None),
                         (100, None),
                         (100, None),
])

food_prices = np.array([
                   1.84,
                   2.19,
                   1.84,
                   1.44,
                   2.29,
                   0.77,
                   1.29,
                   0.60,
                   0.72
])

content_nutri = np.array([
                          (510, 34, 28, 15,  6, 30, 20),
                          (370, 35, 24, 15, 10, 20, 20),
                          (500, 42, 25,  6,  2, 25, 20),
                          (370, 38, 14,  2,  0, 15, 10),
                          (400, 42, 31,  8, 15, 15,  8),
                          (220, 26,  3,  0, 15,  0,  2),
                          (345, 27, 15,  4,  0, 20, 15),
                          (110, 12,  9, 10,  4, 30,  0),
                          ( 80, 20,  1,  2, 120, 2,  2)
])

# 我们来看调整最大热量摄入的不同结果：

for max_calory in [None, 3500, 3000, 2500]:
    intake_thred[0] = (intake_thred[0][0], max_calory)
    
    y = cp.Variable(len(food), boolean=True)
    x = cp.Variable(len(food))
    v = cp.Variable()
    z = cp.Variable(len(nutrients))

    objective = cp.Maximize(cp.sum(y))


    # 营养成分摄入量的约束
    constraints = [ z[j] == cp.sum(x * content_nutri[:, j]) for j in range(len(nutrients))]
    constraints += [ z[j] >= intake_thred[j][0] for j in range(len(nutrients)) if not intake_thred[j][0] is None]
    constraints += [ intake_thred[j][1] >= z[j] for j in range(len(nutrients)) if not intake_thred[j][1] is None]

    # 其他约束
    constraints += [
                  x >= 0,
                  x >= y,
                  v == cp.sum(food_prices * x),
    ]

    # 不想喝太多液体的话，那就加这个约束吧！
    # constraints += [
    #                y[7] + y[8] <= 1
    # ]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()
    print("最大热量摄入： {}卡".format(max_calory))
    print("最多可吃食物种类：{:d}".format(int(result)))
    print("吃的食物：", [fn for choose, fn in zip(y.value, food) if choose])
    print("花钱：${:.2f}".format(v.value))
    print("-"*50)

最大热量摄入： None卡
最多可吃食物种类：9
吃的食物： ['芝士牛肉堡', '马卡龙', '巨无霸', 'FFilet', '鸡肉', '炸薯条', '麦满分', '1%LF Milk', '橙汁']
花钱：$16.47
--------------------------------------------------
最大热量摄入： 3500卡
最多可吃食物种类：9
吃的食物： ['芝士牛肉堡', '马卡龙', '巨无霸', 'FFilet', '鸡肉', '炸薯条', '麦满分', '1%LF Milk', '橙汁']
花钱：$16.30
--------------------------------------------------
最大热量摄入： 3000卡
最多可吃食物种类：6
吃的食物： ['芝士牛肉堡', '马卡龙', '巨无霸', '麦满分', '1%LF Milk', '橙汁']
花钱：$15.75
--------------------------------------------------
最大热量摄入： 2500卡
最多可吃食物种类：3
吃的食物： ['马卡龙', '1%LF Milk', '橙汁']
花钱：$16.67
--------------------------------------------------


### c.选书问题

In [8]:
# data
# !wget https://raw.githubusercontent.com/AnnaNican/optimizers/master/goodreads_bestsellers.csv
# !ls

In [9]:
df = pd.read_csv('goodreads_bestsellers.csv')
book_titles = df.book_name
ratings = df.iloc[:, 7].values
pages = df.pages.values

gamma = 0.1

x = cp.Variable(df.shape[0], boolean=True)
rx = ratings * x
objective = cp.Maximize(cp.sum(x) * gamma + cp.sum(rx) * (1 - gamma))

constraints = [
        cp.sum(pages * x) <= 60 * 5 * 52
]

prob = cp.Problem(objective, constraints)

result = prob.solve()
print("最多可看{:d}本".format(int(np.sum(x.value))))

最多可看45本


In [10]:
print("看这几本书")
print(book_titles[x.value == 1])

看这几本书
1                                The Fault in Our Stars
2                                       The Nightingale
3                                 The Girl on the Train
5                                          Orphan Train
9                                The Invention of Wings
10                                        Me Before You
12                                          Sarah's Key
13                                          Before I Go
16    As Chimney Sweepers Come to Dust (Flavia de Lu...
21                                       Station Eleven
22    Be Careful What You Wish For (The Clifton Chro...
23                         Memory Man (Amos Decker, #1)
24                  The Burning Room (Harry Bosch, #19)
25                   Obsession in Death (In Death, #40)
29                   Last to Die (Rizzoli & Isles, #10)
30                    First Frost (Waverley Family, #2)
33    Mightier Than the Sword (The Clifton Chronicle...
36                                      Th